In [1]:
from IGraph import *
from AbstractGraph import *
from UGraphModel import *
from DGraphModel import *

In [2]:
graph = UGraphModel()
graph.add("HCM")
graph.add("LongAn")
graph.add("BinhDuong")
graph.add("DongNai")
graph.add("BinhThuan")
graph.connect("HCM", "LongAn")
graph.connect("HCM", "BinhDuong")
graph.connect("HCM", "DongNai")
graph.connect("DongNai", "BinhThuan")
graph.println()

Vertices:
 V(HCM, in:3, out:3)
 V(LongAn, in:1, out:1)
 V(BinhDuong, in:1, out:1)
 V(DongNai, in:2, out:2)
 V(BinhThuan, in:1, out:1)
------------------------------------------------------
Edges:
 E(HCM, LongAn, 0)
 E(HCM, BinhDuong, 0)
 E(HCM, DongNai, 0)
 E(LongAn, HCM, 0)
 E(BinhDuong, HCM, 0)
 E(DongNai, HCM, 0)
 E(DongNai, BinhThuan, 0)
 E(BinhThuan, DongNai, 0)



In [3]:
model = UGraphModel()
print("Build Model")
model.add('A')
model.add('B')
model.add('C')
model.add('D')
model.add('E')
model.add('F')
model.connect('A', 'B', 6)
model.connect('A', 'C', 3)

model.connect('B', 'C', 2)
model.connect('B', 'D', 5)

model.connect('C', 'D', 3)
model.connect('C', 'E', 4)

model.connect('D', 'E', 2)
model.connect('D', 'F', 3)

model.connect('E', 'F', 5)

model.println()

print("Disconnect C with remaining vertexes")
model.disconnect('C', 'A')
model.disconnect('C', 'B')
model.disconnect('C', 'D')
model.disconnect('C', 'E')

model.println()

print("Remove vertex A")
model.remove('A')
model.println()

Build Model
Vertices:
 V(A, in:2, out:2)
 V(B, in:3, out:3)
 V(C, in:4, out:4)
 V(D, in:4, out:4)
 V(E, in:3, out:3)
 V(F, in:2, out:2)
------------------------------------------------------
Edges:
 E(A, B, 6)
 E(A, C, 3)
 E(B, A, 6)
 E(B, C, 2)
 E(B, D, 5)
 E(C, A, 3)
 E(C, B, 2)
 E(C, D, 3)
 E(C, E, 4)
 E(D, B, 5)
 E(D, C, 3)
 E(D, E, 2)
 E(D, F, 3)
 E(E, C, 4)
 E(E, D, 2)
 E(E, F, 5)
 E(F, D, 3)
 E(F, E, 5)

Disconnect C with remaining vertexes
Vertices:
 V(A, in:1, out:1)
 V(B, in:2, out:2)
 V(C, in:0, out:0)
 V(D, in:3, out:3)
 V(E, in:2, out:2)
 V(F, in:2, out:2)
------------------------------------------------------
Edges:
 E(A, B, 6)
 E(B, A, 6)
 E(B, D, 5)
 E(D, B, 5)
 E(D, E, 2)
 E(D, F, 3)
 E(E, D, 2)
 E(E, F, 5)
 E(F, D, 3)
 E(F, E, 5)

Remove vertex A
Vertices:
 V(B, in:1, out:1)
 V(C, in:0, out:0)
 V(D, in:3, out:3)
 V(E, in:2, out:2)
 V(F, in:2, out:2)
------------------------------------------------------
Edges:
 E(B, D, 5)
 E(D, B, 5)
 E(D, E, 2)
 E(D, F, 3)
 E(E, D, 2

In [4]:
grapD = DGraphModel()
for idx in range(10):
    grapD.add(str(idx))

grapD.connect('0', '1')
grapD.connect('0', '5')
grapD.connect('1', '7')
grapD.connect('3', '2')
grapD.connect('3', '4')
grapD.connect('3', '7')
grapD.connect('3', '8')
grapD.connect('4', '8')
grapD.connect('6', '0')
grapD.connect('6', '1')
grapD.connect('6', '2')
grapD.connect('8', '2')
grapD.connect('8', '7')
grapD.connect('9', '4')
grapD.println()

Vertices:
 V(0, in:1, out:2)
 V(1, in:2, out:1)
 V(2, in:3, out:0)
 V(3, in:0, out:4)
 V(4, in:2, out:1)
 V(5, in:1, out:0)
 V(6, in:0, out:3)
 V(7, in:3, out:0)
 V(8, in:2, out:2)
 V(9, in:0, out:1)
------------------------------------------------------
Edges:
 E(0, 1, 0)
 E(0, 5, 0)
 E(1, 7, 0)
 E(3, 2, 0)
 E(3, 4, 0)
 E(3, 7, 0)
 E(3, 8, 0)
 E(4, 8, 0)
 E(6, 0, 0)
 E(6, 1, 0)
 E(6, 2, 0)
 E(8, 2, 0)
 E(8, 7, 0)
 E(9, 4, 0)



In [5]:
model = UGraphModel()
print("Build model")
model.add('A')
model.add('B')
model.add('C')
model.add('D')
model.add('E')
model.add('F')
model.connect('A', 'B', 6)
model.connect('A', 'C', 3)

model.connect('B', 'C', 2)
model.connect('B', 'D', 5)

model.connect('C', 'D', 3)
model.connect('C', 'E', 4)

model.connect('D', 'E', 2)
model.connect('D', 'F', 3)

model.connect('E', 'F', 5)

model.println()

print("Disconnect vertex C with remaining vertexes")
model.disconnect('C', 'A')
model.disconnect('C', 'B')
model.disconnect('C', 'D')
model.disconnect('C', 'E')
model.println()

print("Remove vertex A")
model.remove('A')
model.println()

Build model
Vertices:
 V(A, in:2, out:2)
 V(B, in:3, out:3)
 V(C, in:4, out:4)
 V(D, in:4, out:4)
 V(E, in:3, out:3)
 V(F, in:2, out:2)
------------------------------------------------------
Edges:
 E(A, B, 6)
 E(A, C, 3)
 E(B, A, 6)
 E(B, C, 2)
 E(B, D, 5)
 E(C, A, 3)
 E(C, B, 2)
 E(C, D, 3)
 E(C, E, 4)
 E(D, B, 5)
 E(D, C, 3)
 E(D, E, 2)
 E(D, F, 3)
 E(E, C, 4)
 E(E, D, 2)
 E(E, F, 5)
 E(F, D, 3)
 E(F, E, 5)

Disconnect vertex C with remaining vertexes
Vertices:
 V(A, in:1, out:1)
 V(B, in:2, out:2)
 V(C, in:0, out:0)
 V(D, in:3, out:3)
 V(E, in:2, out:2)
 V(F, in:2, out:2)
------------------------------------------------------
Edges:
 E(A, B, 6)
 E(B, A, 6)
 E(B, D, 5)
 E(D, B, 5)
 E(D, E, 2)
 E(D, F, 3)
 E(E, D, 2)
 E(E, F, 5)
 E(F, D, 3)
 E(F, E, 5)

Remove vertex A
Vertices:
 V(B, in:1, out:1)
 V(C, in:0, out:0)
 V(D, in:3, out:3)
 V(E, in:2, out:2)
 V(F, in:2, out:2)
------------------------------------------------------
Edges:
 E(B, D, 5)
 E(D, B, 5)
 E(D, E, 2)
 E(D, F, 3)
 E(

In [6]:
model = DGraphModel()
for idx in range(10):
    model.add(str(idx))

model.connect('0', '1')
model.connect('0', '5')
model.connect('1', '7')
model.connect('3', '2')
model.connect('3', '4')
model.connect('3', '7')
model.connect('3', '8')
model.connect('4', '8')
model.connect('6', '0')
model.connect('6', '1')
model.connect('6', '2')
model.connect('8', '2')
model.connect('8', '7')
model.connect('9', '4')
model.println()

Vertices:
 V(0, in:1, out:2)
 V(1, in:2, out:1)
 V(2, in:3, out:0)
 V(3, in:0, out:4)
 V(4, in:2, out:1)
 V(5, in:1, out:0)
 V(6, in:0, out:3)
 V(7, in:3, out:0)
 V(8, in:2, out:2)
 V(9, in:0, out:1)
------------------------------------------------------
Edges:
 E(0, 1, 0)
 E(0, 5, 0)
 E(1, 7, 0)
 E(3, 2, 0)
 E(3, 4, 0)
 E(3, 7, 0)
 E(3, 8, 0)
 E(4, 8, 0)
 E(6, 0, 0)
 E(6, 1, 0)
 E(6, 2, 0)
 E(8, 2, 0)
 E(8, 7, 0)
 E(9, 4, 0)



In [7]:
DFS = 0
BFS = 1
sorter = TopoSorter(model)
topo = sorter.sort(BFS)
print("topoBFS:", topo)

topo = sorter.sort(DFS)
print("topoDFS:", topo)

topoBFS: ['3', '6', '9', '0', '4', '1', '5', '8', '2', '7']
topoDFS: ['3', '6', '0', '5', '1', '9', '4', '8', '2', '7']


In [8]:
model = UGraphModel()
model.add('A')
model.add('B')
model.add('C')
model.add('D')
model.add('E')
model.add('F')
model.connect('A', 'B', 6)
model.connect('A', 'C', 3)

model.connect('B', 'C', 2)
model.connect('B', 'D', 5)

model.connect('C', 'D', 3)
model.connect('C', 'E', 4)

model.connect('D', 'E', 2)
model.connect('D', 'F', 3)

model.connect('E', 'F', 5)

model.println()

model.disconnect('C', 'A')
model.disconnect('C', 'B')
model.disconnect('C', 'D')
model.disconnect('C', 'E')

model.println()

mst = UGraphModelAlgorithm(model) 
tree = mst.minSpanningTree()
tree.println()

Vertices:
 V(A, in:2, out:2)
 V(B, in:3, out:3)
 V(C, in:4, out:4)
 V(D, in:4, out:4)
 V(E, in:3, out:3)
 V(F, in:2, out:2)
------------------------------------------------------
Edges:
 E(A, B, 6)
 E(A, C, 3)
 E(B, A, 6)
 E(B, C, 2)
 E(B, D, 5)
 E(C, A, 3)
 E(C, B, 2)
 E(C, D, 3)
 E(C, E, 4)
 E(D, B, 5)
 E(D, C, 3)
 E(D, E, 2)
 E(D, F, 3)
 E(E, C, 4)
 E(E, D, 2)
 E(E, F, 5)
 E(F, D, 3)
 E(F, E, 5)

Vertices:
 V(A, in:1, out:1)
 V(B, in:2, out:2)
 V(C, in:0, out:0)
 V(D, in:3, out:3)
 V(E, in:2, out:2)
 V(F, in:2, out:2)
------------------------------------------------------
Edges:
 E(A, B, 6)
 E(B, A, 6)
 E(B, D, 5)
 E(D, B, 5)
 E(D, E, 2)
 E(D, F, 3)
 E(E, D, 2)
 E(E, F, 5)
 E(F, D, 3)
 E(F, E, 5)

vertexList: ['A', 'B', 'C', 'D', 'E', 'F']
parent:A, child:B, weight:6
smallest vFrom:A, vTo:B, weight:6
parent:B, child:D, weight:5
smallest vFrom:B, vTo:D, weight:5
parent:D, child:E, weight:2
parent:D, child:F, weight:3
smallest vFrom:D, vTo:E, weight:2
parent:D, child:F, weight:3
paren